## Music Composition Using Quantum Annealing

Original Code: Özlem Salehi (osalehi@iitis.pl)

Adapted to run on Dynex by: Y3TI & Sam Rehmeh

---

### Installation

In order to run the codes, you need to install the following packages:
- Ocean Software
- pickle

To install Ocean Software, you may use the command `pip install dwave-ocean-sdk`. You can visit [Ocean](https://docs.ocean.dwavesys.com/en/stable/overview/install.html) webpage for further information. 

After the installation, follow the guidelines in the Ocean webpage to configure your system. Depending on your country of residence, you may not be able to access D-Wave machines. Nevertheless, you can still run the codes using simulated annealer.

We will use the [pickle](https://docs.python.org/3.8/library/pickle.html) module to save data. You can install it by the command `pip install pickle` or omit it if you don't want to save the outputs.


### Imports

You need to import the following to run the codes. If you have made a successfull installation, you should be able to run the following cell without any error. 

In [2]:
# D-Wave modules
import dimod
import neal
from dwave.system.samplers.leap_hybrid_sampler import LeapHybridSampler
from dwave.system import EmbeddingComposite, DWaveSampler
import dwave_networkx as dnx 
import dynex
# Pyqubo will be used formulate the models
from pyqubo import Binary, Constraint, LogEncInteger

# pickle will be used for saving data
import pickle

# This is a built in module which we will use in the header of the for loops.
from itertools import product

### Helper functions 

We will list some functions that will be used in the rest of the notebook.

<b>Storing results</b>

The sampleset returned by D-Wave is converted in to a serializable object which is convenient for storing and pickle is used to store it.

In [3]:
def store_results(filename, sampleset):
    """Stores the sampleset obtained from D-Wave
    
    :param filename: name of the file 
    :type filename: string
    :param sampleset: sampleset to be stored
    :type sampleset: dimod.SampleSet
    """
    sdf = sampleset.to_serializable()
    with open(filename, 'wb') as handle:
        pickle.dump(sdf, handle)

<b> Annealing experiment</b>

This function is used to perform the annealing experiment. Annealing type is either 's', 'h' or 'q', representing simulated, hybrid and quantum respectively. The parameter `ns` which stands for number of sweeps is needed only for simulated annealing. `at` is the annealing time for quantum annealing and `cs` determines the chain strength. For hybrid annealing, no parameters are used. If any of the parameters are not provided, then their default values are used.

In [4]:
def anneal(bqm, anneal_type, ns=1000, nr=1000, at=20, cs=1):
    """Runs an annealing experiment and returns the sampleset
    
    :param bqm: binary quadratic model to be sampled
    :type bqm: BinaryQuadraticModel
    :param type: type of the annealing (simulated, hybrid, quantum)
    :type: string
    :param ns: Number of sweeps
    :type ns: int
    :param nr: Number of samples
    :type nr: int
    :param at: Annealing time
    :type at: int
    :param cs: Chain strength 
    :type cs: float
    :return: sampleset
    :rtype: dimod.SampleSet
    """
    if anneal_type == 's':
        s = neal.SimulatedAnnealingSampler()
        sampleset = s.sample(bqm, beta_range=(5, 100), num_sweeps=ns, num_reads=ns,
                             beta_schedule_type='geometric')
    elif anneal_type == 'h':
        s = LeapHybridSampler()
        sampleset = s.sample(bqm)
    elif anneal_type == 'q':
        s = EmbeddingComposite(DWaveSampler())
        sampleset = s.sample(bqm, chain_strength = cs, annealing_time = at, num_reads=1000)

    results, energies = [], []

    for datum in sampleset.data(fields=["sample", "energy"]):
        energies.append(datum.energy)
        results.append(datum[0])
    return sampleset

### DYNEX Experiment

In [24]:
def DYNEX(bqm, num_reads=10000, annealing_time=1000):
    model = dynex.BQM(bqm)
    sampler = dynex.DynexSampler(model, mainnet=False, description='Music Composition', bnb=False)
    sampleset = sampler.sample(num_reads=100000, annealing_time = 1000, debugging=False)
    solution = sampleset.first.sample
    energy = sampleset.first.energy
    return solution, energy

<b> Converting sample into a pitch sequence </b>

The output we get from D-Wave is not a pitch sequence but a list of binary variables and their values. After getting this output, we need to convert it into a pitch sequence.

In [25]:
def translate_sample(sample, P, n):
    """Converts a given sample into a pitch sequence
     
    :param sample: A dictionary representing a sample where keys are the variable names and values are either 0 or 1
    :param type sample: dict
    :param P: List of the pitches
    :param type P: list
    :param n: Number of notes in the sequence
    :param type n: int
    :return: sequence of pitches
    :rtype: list
    """
    seq = []
    for i,p in product(range(1,n+1),P):
        if sample[f"x_{i}_{p}"]==1:
            seq.append(p)
    return seq

<b> Converting sample into a chord sequence

In case we would like to create a chord sequence, there will be three notes at each time point. The following function creates a chord sequence from the sample.

In [26]:
from itertools import product

def translate_chord(sample,P,n):
    """Converts a given sample into a chord sequence
     
    :param sample: A dictionary representing a sample where keys are the variable names and values are either 0 or 1
    :param type sample: dict
    :param P: List of the pitches
    :param type P: list
    :param n: Number of notes in the sequence
    :param type n: int
    :return: sequence of chords
    :rtype: list
    """
    seq = {i:[] for i in range(1,n+1)}
    for i,p in product(range(1,n+1),P):
        if sample[f"x_{i}_{p}"]==1:
            seq[i].append(p)
    return seq

<b> Checking if constraints are broken using pyqubo

pyqubo has a built-in method to check whether the constraints defined using the `Constraint` keyword are broken or not. (It does not detect if there are penalty values resulting from the terms directly added to the objective.)

In [27]:
def check_constraints(sample, model):    
    """Prints the violated constraints in a given sample
     
    :param sample: A dictionary representing a sample where keys are the variable names and values are either 0 or 1
    :param type sample: dict
    :param model: Model that is representing the qubo problem
    :param type P: Pyqubo model
    """
    dec = model.decode_sample(sample, vartype='BINARY')
    print(dec.constraints(only_broken=True))

----

### Example 1 (Page 13) - Basic model for melody generation

<b> Creating the model

We first create a list of pitches `P` and set `n`, which is the number of notes in the music piece to be generated. 

In [141]:
P = ['C4', 'D4', 'E4','G4' ]
n = 5

Next we add the single note constraint, which is defined as 
$
\left ( 1- \sum_{j \in P} x_{i,j} \right )^2
$
for each $i=1,\dots,5$. (Eq. 18)

We use pyqubo to formulate the model. 

- $H$ will represent our QUBO formulation.
- Note that we have a for loop going through each $i$ and we add the constraint corresponding to $i$ to $H$.
- `Binary(x)` keyword defines a binary variable with name `x`
-  `sum(Binary(f"x_{i}_{j}") for j in P)` represents the sum $\sum_{j \in P} x_{i,j}$
- `Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")` adds the constraint $
\left ( 1- \sum_{j \in P} x_{i,j} \right )^2
$ to the model with the name `Single_note_{i}`


In [142]:
H = 0
for i in range(1,n+1):
    H += Constraint((1-sum(Binary(f"x_{i}_{j}") for j in P))**2, f"Single_note_{i}")

We can print `H` to see what it looks like.

In [143]:
H

(Constraint(((1.000000 + (-1.000000 * (Binary('x_5_G4') + Binary('x_5_E4') + Binary('x_5_D4') + 0.000000 + Binary('x_5_C4')))) * (1.000000 + (-1.000000 * (Binary('x_5_G4') + Binary('x_5_E4') + Binary('x_5_D4') + 0.000000 + Binary('x_5_C4'))))), 'Single_note_5') + Constraint(((1.000000 + (-1.000000 * (Binary('x_4_G4') + Binary('x_4_E4') + Binary('x_4_D4') + 0.000000 + Binary('x_4_C4')))) * (1.000000 + (-1.000000 * (Binary('x_4_G4') + Binary('x_4_E4') + Binary('x_4_D4') + 0.000000 + Binary('x_4_C4'))))), 'Single_note_4') + Constraint(((1.000000 + (-1.000000 * (Binary('x_3_G4') + Binary('x_3_E4') + Binary('x_3_D4') + 0.000000 + Binary('x_3_C4')))) * (1.000000 + (-1.000000 * (Binary('x_3_G4') + Binary('x_3_E4') + Binary('x_3_D4') + 0.000000 + Binary('x_3_C4'))))), 'Single_note_3') + Constraint(((1.000000 + (-1.000000 * (Binary('x_2_G4') + Binary('x_2_E4') + Binary('x_2_D4') + 0.000000 + Binary('x_2_C4')))) * (1.000000 + (-1.000000 * (Binary('x_2_G4') + Binary('x_2_E4') + Binary('x_2_D4') +

<b> Running the experiment

To run an annealing experiment using Ocean SDK, we need a BinaryQuadraticModel.

First we use pyqubo's `compile` function to generate the `model` object, which is then converted into a QUBO using `to_qubo()` function of pyqubo. 

From `qubo` and `of` representing the QUBO model and the offset respectively, we create a BinaryQuadraticModel object of D-Wave.

In [144]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

Let's print `bqm` as well.

In [145]:
bqm

BinaryQuadraticModel({'x_1_G4': -1.0, 'x_1_D4': -1.0, 'x_3_D4': -1.0, 'x_5_C4': -1.0, 'x_5_D4': -1.0, 'x_4_G4': -1.0, 'x_4_C4': -1.0, 'x_4_E4': -1.0, 'x_5_G4': -1.0, 'x_4_D4': -1.0, 'x_5_E4': -1.0, 'x_3_G4': -1.0, 'x_3_E4': -1.0, 'x_3_C4': -1.0, 'x_2_D4': -1.0, 'x_2_E4': -1.0, 'x_2_C4': -1.0, 'x_1_C4': -1.0, 'x_2_G4': -1.0, 'x_1_E4': -1.0}, {('x_1_D4', 'x_1_G4'): 2.0, ('x_5_D4', 'x_5_C4'): 2.0, ('x_4_C4', 'x_4_G4'): 2.0, ('x_4_E4', 'x_4_G4'): 2.0, ('x_4_E4', 'x_4_C4'): 2.0, ('x_5_G4', 'x_5_C4'): 2.0, ('x_5_G4', 'x_5_D4'): 2.0, ('x_4_D4', 'x_4_G4'): 2.0, ('x_4_D4', 'x_4_C4'): 2.0, ('x_4_D4', 'x_4_E4'): 2.0, ('x_5_E4', 'x_5_C4'): 2.0, ('x_5_E4', 'x_5_D4'): 2.0, ('x_5_E4', 'x_5_G4'): 2.0, ('x_3_G4', 'x_3_D4'): 2.0, ('x_3_E4', 'x_3_D4'): 2.0, ('x_3_E4', 'x_3_G4'): 2.0, ('x_3_C4', 'x_3_D4'): 2.0, ('x_3_C4', 'x_3_G4'): 2.0, ('x_3_C4', 'x_3_E4'): 2.0, ('x_2_E4', 'x_2_D4'): 2.0, ('x_2_C4', 'x_2_D4'): 2.0, ('x_2_C4', 'x_2_E4'): 2.0, ('x_1_C4', 'x_1_G4'): 2.0, ('x_1_C4', 'x_1_D4'): 2.0, ('x_2_G4

The next step is to run the experiment. We use the `anneal` function we have previously written. In this example, we select quantum annealing by providing the parameter `q`. As we don't provide any additional parameters, default parameters will be used for annealing time and chain strength.

<i>Note: If you did not configure D-Wave access or you can not configure it due to your country of residence, then you can use 's' keyword to use simulated annealer instead. 

#### Let's quantum-compute it on DYNEX

In [146]:
sampleset, energy = DYNEX(bqm)

[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬───────────────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CIRCUITS   │ STEPS   │ GROUND STATE   │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼───────────────────────────┼────────────┼─────────┼────────────────┤
│         -1 │       28 │              58 │           0 │           │ *** WAITING FOR READS *** │            │         │                │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴───────────────────────────┴────────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.31 SECONDS
[DYNEX] SAMPLESET READY


<b> Interpreting the results

Let's see what samples look like. 

In [147]:
print(sampleset, energy)

{'x_1_C4': 0, 'x_1_D4': 0, 'x_1_E4': 1, 'x_1_G4': 0, 'x_2_C4': 0, 'x_2_D4': 0, 'x_2_E4': 0, 'x_2_G4': 1, 'x_3_C4': 0, 'x_3_D4': 0, 'x_3_E4': 1, 'x_3_G4': 0, 'x_4_C4': 0, 'x_4_D4': 1, 'x_4_E4': 0, 'x_4_G4': 0, 'x_5_C4': 0, 'x_5_D4': 1, 'x_5_E4': 0, 'x_5_G4': 0} 0.0


When you consider a line from the samples list, the list is the value of the binary variables, 0 indicates that the energy is 0, and 1 indicates that this specific sample is observed only once.
`[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0], 0., 1)`

The order of the variables can be seen from the list of variables.

`Variables(['x_1_C4', 'x_1_D4', 'x_1_E4', 'x_1_G4', 'x_2_C4', 'x_2_D4', 'x_2_E4', 'x_2_G4', 'x_3_C4', 'x_3_D4', 'x_3_E4', 'x_3_G4', 'x_4_C4', 'x_4_D4', 'x_4_E4', 'x_4_G4', 'x_5_C4', 'x_5_D4', 'x_5_E4', 'x_5_G4']`

As the lowest energy sample has energy 0, we expect that no constraint is violated. We can check it also using pyqubo. The returned dictionary will contain the list of violated constraints, or {} in case no constraint is violated. 

In [35]:
check_constraints(sampleset, model) 

{}


If you want to store the results, you can run the following cell. 

In [38]:
filename ="experiment1"
solution = dimod.SampleSet.from_samples(sampleset, energy=energy, vartype=dimod.BINARY)
store_results(filename, solution)

---

### Example 2 (page 14) - Avoiding certain consecutive notes

<b> Creating the model

We use the same `P` and `n` as before.

In [148]:
P = ['C4', 'D4', 'E4','G4' ]
n = 5

First we define the single note constraint. (Eq. 18)

In [149]:
H = 0
for i in range(1,n+1):
    H += Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We want to enforce that G4 does not follow D4. We add the term ` Binary(f"x_{i}_D4")*Binary(f"x_{i+1}_G4")` directly to the objective for each $i$. Note that the penalty coefficient is 1 in this case. (Eq.20)

In [150]:
for i in range(1,n+1):
    H += Binary(f"x_{i}_D4")*Binary(f"x_{i+1}_G4")     

We will force that the same note does not appear more than twice in a row. (Eq. 22)

$ x_{i,j} + x_{i+1,j} + x_{i+2,j} \leq 2$

As we have an inequality, we need to introduce a slack variable $s_{i,j}$. 

$ x_{i,j} + x_{i+1,j} + x_{i+2,j} +s_{i,j} = 2$

The upper bound for the slack variable is 2, as all three binary variables can be equal to 0 simulatenously. Hence we need an integer variable $s$. We use `LogEncInteger(f"s_{i}_{j}",(0,2))` from pyqubo to create an integer variable named `s_i_j`, with the lower and upper bounds (0,2). 

Finally, we convert it to the following constraint add to our model.

$ (x_{i,j} + x_{i+1,j} + x_{i+2,j} +s_{i,j} - 2)^2$

Notice that the penalty value is set as 1.

In [151]:
for i in range(1,n-1):
    for j in P:
        H += Constraint((Binary(f"x_{i+2}_{j}") + Binary(f"x_{i}_{j}") + Binary(f"x_{i+1}_{j}") + LogEncInteger(f"s_{i}_{j}",(0,2))-2)**2, f"same_note_{i}_{j}")

<b> Running the experiment

In [152]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

We increase the chain strength to 1.5 this time.

In [153]:
#sampleset = anneal(bqm, 's', cs=1.5)
sampleset, energy = DYNEX(bqm)

[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬───────────────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CIRCUITS   │ STEPS   │ GROUND STATE   │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼───────────────────────────┼────────────┼─────────┼────────────────┤
│         -1 │       53 │             199 │           0 │           │ *** WAITING FOR READS *** │            │         │                │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴───────────────────────────┴────────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.32 SECONDS
[DYNEX] SAMPLESET READY


<b>Interpreting the results

Let's print the latest sample with the lowest energy.

In [154]:
print(sampleset, energy)

{'s_1_C4[0]': 1, 's_1_C4[1]': 0, 's_1_D4[0]': 1, 's_1_D4[1]': 0, 's_1_E4[0]': 1, 's_1_E4[1]': 1, 's_1_G4[0]': 0, 's_1_G4[1]': 1, 's_2_C4[0]': 1, 's_2_C4[1]': 0, 's_2_D4[0]': 1, 's_2_D4[1]': 0, 's_2_E4[0]': 1, 's_2_E4[1]': 0, 's_2_G4[0]': 1, 's_2_G4[1]': 1, 's_3_C4[0]': 1, 's_3_C4[1]': 0, 's_3_D4[0]': 1, 's_3_D4[1]': 1, 's_3_E4[0]': 0, 's_3_E4[1]': 0, 's_3_G4[0]': 1, 's_3_G4[1]': 1, 'x_1_C4': 0, 'x_1_D4': 0, 'x_1_E4': 0, 'x_1_G4': 1, 'x_2_C4': 0, 'x_2_D4': 1, 'x_2_E4': 0, 'x_2_G4': 0, 'x_3_C4': 1, 'x_3_D4': 0, 'x_3_E4': 0, 'x_3_G4': 0, 'x_4_C4': 0, 'x_4_D4': 0, 'x_4_E4': 1, 'x_4_G4': 0, 'x_5_C4': 0, 'x_5_D4': 0, 'x_5_E4': 1, 'x_5_G4': 0, 'x_6_G4': 1} 0.0


In [155]:
filename = "experiment2"
solution = dimod.SampleSet.from_samples(sampleset, energy=energy, vartype=dimod.BINARY)
store_results(filename, solution)

---

### Example 3 (Page 14) - Semitones and avoiding augmented intervals

 <b> Creating the model

We select the subset of the chromatic scale as `P` and we will generate 20 notes.

In [156]:
P= [0,2,4,5,7,9,11,12]
n = 20

`A` is the list of unallowed intervals.

In [157]:
A= [6,8,10,12]

This time, we will set some penalty values to be  different from 1.

In [158]:
P1, P2, P3, P4 = 20,2,1,1

We start with the single note constraint. (Eq.18)

In [159]:
H = 0
for i in range(1,n+1):
    H += Constraint(P1*(1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We include the rule about no more than two same note in a sequence. (Eq.22)

In [160]:
for i in range(1,n-1):
    for j in P:
        H += Constraint(P2*(Binary(f"x_{i+2}_{j}") + Binary(f"x_{i}_{j}") + Binary(f"x_{i+1}_{j}") + LogEncInteger(f"s_{i}_{j}",(0,2))-2)**2, f"same_note_{i}_{j}")

We add penalty terms to the objective for the unallowed intervals, by adding the following term: $   x_{i,j}x_{i+1,j'}  $
for all $i \in [n], |j'-j| \in A$. (Eq.24) 



In [161]:
for i in range(1,n):
    for j, k in product(P,P):
        if abs(j-k) in A:
            H += P3*Binary(f"x_{i}_{j}")*(Binary(f"x_{i+1}_{k}"))

We add the rule about the first and the last notes by adding the following terms to the objective: $(1-x_{1,0}),~~(1-x_{n,0})$. (Eq.26)

In [162]:
H += P4*(1-Binary(f"x_1_0")) + P4*(1-Binary(f"x_20_0"))   

<b> Running the experiment

In [163]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

We will use Dynex Neuromorphic Computing but with higher number of reads

In [164]:
#sampleset = anneal(bqm, 'h')
sampleset, energy = DYNEX(bqm, num_reads=100000, annealing_time=1000)

[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬───────────────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CIRCUITS   │ STEPS   │ GROUND STATE   │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼───────────────────────────┼────────────┼─────────┼────────────────┤
│         -1 │      456 │            2472 │           0 │           │ *** WAITING FOR READS *** │            │         │                │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴───────────────────────────┴────────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.30 SECONDS
[DYNEX] SAMPLESET READY


Dynex returns only a single solution. (lowest energy)

In [165]:
print(sampleset, energy)

{'s_10_0[0]': 1, 's_10_0[1]': 1, 's_10_11[0]': 0, 's_10_11[1]': 0, 's_10_12[0]': 1, 's_10_12[1]': 1, 's_10_2[0]': 1, 's_10_2[1]': 1, 's_10_4[0]': 1, 's_10_4[1]': 1, 's_10_5[0]': 1, 's_10_5[1]': 1, 's_10_7[0]': 0, 's_10_7[1]': 0, 's_10_9[0]': 1, 's_10_9[1]': 1, 's_11_0[0]': 1, 's_11_0[1]': 0, 's_11_11[0]': 1, 's_11_11[1]': 0, 's_11_12[0]': 1, 's_11_12[1]': 1, 's_11_2[0]': 1, 's_11_2[1]': 1, 's_11_4[0]': 1, 's_11_4[1]': 1, 's_11_5[0]': 1, 's_11_5[1]': 1, 's_11_7[0]': 0, 's_11_7[1]': 1, 's_11_9[0]': 1, 's_11_9[1]': 1, 's_12_0[0]': 0, 's_12_0[1]': 1, 's_12_11[0]': 1, 's_12_11[1]': 0, 's_12_12[0]': 1, 's_12_12[1]': 1, 's_12_2[0]': 1, 's_12_2[1]': 0, 's_12_4[0]': 1, 's_12_4[1]': 1, 's_12_5[0]': 1, 's_12_5[1]': 1, 's_12_7[0]': 1, 's_12_7[1]': 1, 's_12_9[0]': 1, 's_12_9[1]': 1, 's_13_0[0]': 0, 's_13_0[1]': 1, 's_13_11[0]': 1, 's_13_11[1]': 1, 's_13_12[0]': 1, 's_13_12[1]': 1, 's_13_2[0]': 1, 's_13_2[1]': 0, 's_13_4[0]': 1, 's_13_4[1]': 1, 's_13_5[0]': 1, 's_13_5[1]': 1, 's_13_7[0]': 1, 's_13_7

The following code provided by pyqubo can be used to identify the constraints that are broken (if any). You can try running the same experiment by trying quantum annealing directly and check if some constraints are violated or not. What you can try when some constraint is violated is to increase its penalty value, and overall increase the annealing time and number of reads. Still, you may not be able to get a sample in which all constraints are satisfied. In such a case, you can run hybrid solver.

In [58]:
dec = model.decode_sample(sampleset, vartype='BINARY')
print(dec.constraints(only_broken=True))

{'same_note_6_4': (False, 2.0), 'same_note_1_9': (False, 2.0), 'same_note_6_0': (False, 2.0), 'same_note_15_9': (False, 2.0), 'same_note_3_9': (False, 2.0), 'same_note_9_9': (False, 2.0), 'same_note_15_7': (False, 2.0), 'same_note_1_0': (False, 2.0), 'same_note_18_2': (False, 2.0)}


In [59]:
filename = "experiment4"
solution = dimod.SampleSet.from_samples(sampleset, energy=energy, vartype=dimod.BINARY)
store_results(filename, solution)

---

### Example 4 (Page 15) - Diatonic scale and tendency notes

<b>Creating the model

This time, we take the diatonic scale and select `P` as follows. We will generate 20 notes.

In [60]:
P = [1,2,3,4,5,6,7,8]
n = 20

We start by adding the single note constraint. (Eq. 18)

In [61]:
H = 0
for i in range(1,n+1):
    H += Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We initialize the first and the last note of the sequence. (Eq. 26)

In [62]:
H += (1-Binary(f"x_1_1")) + (1-Binary(f"x_20_1"))

Next we add the rules about tendency notes (Eq.28), by adding the following terms directly to the objective.

In [63]:
for i in range(1,n):
    H += Binary(f"x_{i}_2")*(1-Binary(f"x_{i+1}_1"))
    H += Binary(f"x_{i}_4")*(1-Binary(f"x_{i+1}_3"))
    H += Binary(f"x_{i}_6")*(1-Binary(f"x_{i+1}_5"))
    H += Binary(f"x_{i}_7")*(1-Binary(f"x_{i+1}_8"))

<b> Running the experiment

In [64]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

In [66]:
#cs = 1.5
#sampleset = anneal(bqm, 'q', cs)
sampleset, energy = DYNEX(bqm, num_reads=100000, annealing_time=1000)

[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬───────────────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CIRCUITS   │ STEPS   │ GROUND STATE   │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼───────────────────────────┼────────────┼─────────┼────────────────┤
│         -1 │      168 │             880 │           0 │           │ *** WAITING FOR READS *** │            │         │                │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴───────────────────────────┴────────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.29 SECONDS
[DYNEX] SAMPLESET READY


<b> Interpreting the results

Let's print the first sample and lowest energy.

In [67]:
print(sampleset, energy)

{'x_10_1': 0, 'x_10_2': 0, 'x_10_3': 0, 'x_10_4': 0, 'x_10_5': 1, 'x_10_6': 0, 'x_10_7': 0, 'x_10_8': 0, 'x_11_1': 0, 'x_11_2': 0, 'x_11_3': 0, 'x_11_4': 0, 'x_11_5': 0, 'x_11_6': 1, 'x_11_7': 0, 'x_11_8': 0, 'x_12_1': 0, 'x_12_2': 0, 'x_12_3': 0, 'x_12_4': 0, 'x_12_5': 1, 'x_12_6': 0, 'x_12_7': 0, 'x_12_8': 0, 'x_13_1': 1, 'x_13_2': 0, 'x_13_3': 0, 'x_13_4': 0, 'x_13_5': 0, 'x_13_6': 0, 'x_13_7': 0, 'x_13_8': 0, 'x_14_1': 0, 'x_14_2': 0, 'x_14_3': 0, 'x_14_4': 0, 'x_14_5': 1, 'x_14_6': 0, 'x_14_7': 0, 'x_14_8': 0, 'x_15_1': 0, 'x_15_2': 0, 'x_15_3': 0, 'x_15_4': 0, 'x_15_5': 1, 'x_15_6': 0, 'x_15_7': 0, 'x_15_8': 0, 'x_16_1': 0, 'x_16_2': 0, 'x_16_3': 1, 'x_16_4': 0, 'x_16_5': 0, 'x_16_6': 0, 'x_16_7': 0, 'x_16_8': 0, 'x_17_1': 1, 'x_17_2': 0, 'x_17_3': 0, 'x_17_4': 0, 'x_17_5': 0, 'x_17_6': 0, 'x_17_7': 0, 'x_17_8': 0, 'x_18_1': 0, 'x_18_2': 0, 'x_18_3': 0, 'x_18_4': 0, 'x_18_5': 0, 'x_18_6': 0, 'x_18_7': 0, 'x_18_8': 1, 'x_19_1': 0, 'x_19_2': 0, 'x_19_3': 1, 'x_19_4': 0, 'x_19_5': 0

In [69]:
filename = "experiment4"
solution = dimod.SampleSet.from_samples(sampleset, energy=energy, vartype=dimod.BINARY)
store_results(filename, solution)

---

### Example 5 (Page 16-17) - Objective function

<b>Creating the model

We use the diatonic scale as the list of pitches and we will generate 20 notes.

In [70]:
P = [1,2,3,4,5,6,7,8]
n = 20

We define some penalty values. 

In [71]:
P1,P2,P3,P4,P5 = 5,3,1,1,1

We define the matrix of the weights.

In [72]:
M = [[0 for i in range(9)] for j in range(9)]
M[4][4]=2
M[4][3]=2
M[4][5]=1

M[5][6]=1
M[5][4]=1

M[6][6]=1
M[6][5]=1


M[2][1]=1
M[2][3]=1
M[2][2]=1

M[1][1]=1
M[1][2]=1

As always, we start by single note constraint. (Eq.18)

In [73]:
H = 0
for i in range(1,n+1):
    H += P1*Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We add the rule about the first and the last note. (Eq.26)

In [74]:
H += P2*Constraint(1-Binary(f"x_1_1"),"first") + P2*Constraint(1-Binary(f"x_20_1"),"last")

We include the rule about no more than two same note in a sequence. (Eq.22)

In [75]:
for i in range(1,n-1):
    for j in P:
        H += Constraint(P3*(Binary(f"x_{i+2}_{j}") + Binary(f"x_{i}_{j}") + Binary(f"x_{i+1}_{j}") + LogEncInteger(f"s_{i}_{j}",(0,2))-2)**2, f"same_note_{i}_{j}")

Next we add the rules about tendency notes (Eq.28), by adding the following terms directly to the objective.

In [76]:
for i in range(1,n):
    H += P4*Binary(f"x_{i}_2")*(1-Binary(f"x_{i+1}_1"))
    H += P4*Binary(f"x_{i}_4")*(1-Binary(f"x_{i+1}_3"))
    H += P4*Binary(f"x_{i}_6")*(1-Binary(f"x_{i+1}_5"))
    H += P4*Binary(f"x_{i}_7")*(1-Binary(f"x_{i+1}_8"))

Finally, we add the 'rewards' to our objective function, defined as $
    -\sum_{ \substack{i\in [n-1]  \\ j,j' \in P}} W_{j,j'} x_{i,j} x_{i+1,j'}
$. The weights we use are stored inside `M`. (Eq. 29)



In [77]:
for i in range(1,n+1):
    for j, jp in product(P,P):
        if M[j][jp]!=0:
            H += -P5*M[j][jp]*Binary(f"x_{i}_{j}")*Binary(f"x_{i+1}_{jp}")           

<b> Running the experiment

In [78]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

We will use Dynex Neuromorphic Computing but with higher number of reads

In [79]:
sampleset, energy = DYNEX(bqm, num_reads=100000, annealing_time=1000)

[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬───────────────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CIRCUITS   │ STEPS   │ GROUND STATE   │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼───────────────────────────┼────────────┼─────────┼────────────────┤
│         -1 │      462 │            2867 │           0 │           │ *** WAITING FOR READS *** │            │         │                │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴───────────────────────────┴────────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.35 SECONDS
[DYNEX] SAMPLESET READY


<b> Interpreting the results

In [80]:
print(sampleset, energy)

{'s_10_1[0]': 1, 's_10_1[1]': 1, 's_10_2[0]': 1, 's_10_2[1]': 1, 's_10_3[0]': 0, 's_10_3[1]': 0, 's_10_4[0]': 1, 's_10_4[1]': 1, 's_10_5[0]': 0, 's_10_5[1]': 1, 's_10_6[0]': 1, 's_10_6[1]': 1, 's_10_7[0]': 1, 's_10_7[1]': 1, 's_10_8[0]': 1, 's_10_8[1]': 1, 's_11_1[0]': 1, 's_11_1[1]': 1, 's_11_2[0]': 1, 's_11_2[1]': 1, 's_11_3[0]': 1, 's_11_3[1]': 1, 's_11_4[0]': 0, 's_11_4[1]': 0, 's_11_5[0]': 0, 's_11_5[1]': 1, 's_11_6[0]': 1, 's_11_6[1]': 1, 's_11_7[0]': 1, 's_11_7[1]': 1, 's_11_8[0]': 1, 's_11_8[1]': 1, 's_12_1[0]': 1, 's_12_1[1]': 1, 's_12_2[0]': 1, 's_12_2[1]': 1, 's_12_3[0]': 1, 's_12_3[1]': 1, 's_12_4[0]': 1, 's_12_4[1]': 0, 's_12_5[0]': 1, 's_12_5[1]': 0, 's_12_6[0]': 1, 's_12_6[1]': 1, 's_12_7[0]': 1, 's_12_7[1]': 1, 's_12_8[0]': 1, 's_12_8[1]': 1, 's_13_1[0]': 1, 's_13_1[1]': 1, 's_13_2[0]': 1, 's_13_2[1]': 1, 's_13_3[0]': 1, 's_13_3[1]': 1, 's_13_4[0]': 0, 's_13_4[1]': 0, 's_13_5[0]': 1, 's_13_5[1]': 0, 's_13_6[0]': 1, 's_13_6[1]': 1, 's_13_7[0]': 1, 's_13_7[1]': 1, 's_13_8

In [81]:
dec = model.decode_sample(sampleset, vartype='BINARY')
print(dec.constraints(only_broken=True))

{'same_note_6_5': (False, 1.0), 'same_note_16_4': (False, 1.0), 'same_note_4_5': (False, 1.0), 'same_note_14_5': (False, 1.0), 'same_note_10_3': (False, 1.0), 'same_note_17_4': (False, 1.0), 'same_note_2_3': (False, 1.0), 'same_note_7_3': (False, 1.0), 'same_note_5_5': (False, 1.0), 'same_note_15_3': (False, 1.0), 'same_note_9_4': (False, 1.0), 'same_note_9_5': (False, 1.0), 'same_note_2_2': (False, 1.0), 'same_note_1_1': (False, 1.0), 'last': (False, 1.0), 'same_note_3_2': (False, 1.0), 'same_note_12_4': (False, 1.0), 'same_note_4_2': (False, 1.0), 'same_note_1_5': (False, 1.0), 'same_note_16_3': (False, 1.0), 'same_note_6_3': (False, 1.0), 'same_note_4_3': (False, 1.0), 'same_note_10_4': (False, 1.0)}


In [82]:
filename = "experiment5"
solution = dimod.SampleSet.from_samples(sampleset, energy=energy, vartype=dimod.BINARY)
store_results(filename, solution)

---

### Example 6 (Page 17-18) - Rhythm generation

<b> Creating the model

We identify quarter, dotted quarted, eighth and half notes by 0,1,2,3 respectively. Later on, we will use this `Pdict` to translate our sample back.

In [83]:
Pdict = {0:"Q", 1:"DQ", 2:"E", 3:"H"}
P = [0,1,2,3]
n = 20

We define the matrix of weights for the note durations.

In [84]:
M = [[0 for i in range(4)] for j in range(4)]
M[0][0]=11
M[0][1]=1

M[1][2] = 1
M[2][3]=1

Penalty values.

In [85]:
P1, P2, P3 = 50, 50, 1

We start by single note constraint. (Eq.18)

In [86]:
H = 0
for i in range(1,n+1):
    H += P1*Constraint((1-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Single_note_{i}")

We add the rule forcing that each duration appears at least twice. (Eq.36)

In [87]:
for d in P:
    H += P2*Constraint((2-sum(Binary(f"x_{i}_{d}") for i in range(1,n+1))+LogEncInteger(f"s_{d}",(0,14)))**2, f"At_least_two_{d}")

We add the 'rewards' reflecting the weights for consecutive durations. (Eq. 29)

In [88]:
for i in range(1,n+1):
    for j, jp in product(P,P):
        if M[j][jp]!=0:
            H += P3 * -M[j][jp]*Binary(f"x_{i}_{j}")*Binary(f"x_{i+1}_{jp}")

<b> Running the experiment

In [89]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

We will use Dynex Neuromorphic Computing but with higher number of reads

In [90]:
sampleset, energy = DYNEX(bqm, num_reads=100000, annealing_time=1000)

[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬───────────────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CIRCUITS   │ STEPS   │ GROUND STATE   │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼───────────────────────────┼────────────┼─────────┼────────────────┤
│         -1 │      108 │            2151 │           0 │           │ *** WAITING FOR READS *** │            │         │                │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴───────────────────────────┴────────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.32 SECONDS
[DYNEX] SAMPLESET READY


<b> Interpreting the results

In [102]:
print(sampleset,energy)

{'s_0[0]': 0, 's_0[1]': 0, 's_0[2]': 1, 's_0[3]': 0, 's_1[0]': 0, 's_1[1]': 0, 's_1[2]': 0, 's_1[3]': 0, 's_2[0]': 1, 's_2[1]': 0, 's_2[2]': 1, 's_2[3]': 0, 's_3[0]': 0, 's_3[1]': 0, 's_3[2]': 0, 's_3[3]': 0, 'x_10_0': 0, 'x_10_1': 0, 'x_10_2': 1, 'x_10_3': 0, 'x_11_0': 0, 'x_11_1': 0, 'x_11_2': 1, 'x_11_3': 0, 'x_12_0': 0, 'x_12_1': 0, 'x_12_2': 0, 'x_12_3': 0, 'x_13_0': 0, 'x_13_1': 0, 'x_13_2': 1, 'x_13_3': 0, 'x_14_0': 0, 'x_14_1': 0, 'x_14_2': 1, 'x_14_3': 0, 'x_15_0': 1, 'x_15_1': 0, 'x_15_2': 0, 'x_15_3': 0, 'x_16_0': 1, 'x_16_1': 0, 'x_16_2': 0, 'x_16_3': 0, 'x_17_0': 0, 'x_17_1': 0, 'x_17_2': 0, 'x_17_3': 1, 'x_18_0': 0, 'x_18_1': 0, 'x_18_2': 1, 'x_18_3': 0, 'x_19_0': 1, 'x_19_1': 0, 'x_19_2': 0, 'x_19_3': 0, 'x_1_0': 0, 'x_1_1': 0, 'x_1_2': 0, 'x_1_3': 0, 'x_20_0': 0, 'x_20_1': 0, 'x_20_2': 0, 'x_20_3': 1, 'x_21_0': 1, 'x_21_1': 0, 'x_21_2': 0, 'x_21_3': 0, 'x_2_0': 0, 'x_2_1': 0, 'x_2_2': 1, 'x_2_3': 0, 'x_3_0': 0, 'x_3_1': 1, 'x_3_2': 0, 'x_3_3': 0, 'x_4_0': 0, 'x_4_1': 0,

In [103]:
dec = model.decode_sample(sampleset, vartype='BINARY')
print(dec.constraints(only_broken=True))

{'Single_note_12': (False, 1.0), 'At_least_two_3': (False, 1.0), 'Single_note_1': (False, 1.0)}


In [104]:
filename = f"experiment6"
solution = dimod.SampleSet.from_samples(sampleset, energy=energy, vartype=dimod.BINARY)
store_results(filename, solution)

---

### Example 7 (Page 19-20-21) Markov Random Fields

<b> Creating the model</b>

In order to build a Markov Network it is neccessary to create a dictionary which will contain the information about the potentials relating each pair of binary random variables. We want to have a dictionary with the following format:

```
potentials = {('a', 'b'): {(0, 0): 1,  # for when a = 0, b = 0
                          (0, 1): .5,  # for when a = 0, b = 1
                          (1, 0): .5,  # for when a = 1, b = 0
                          (1, 1): 2}}  # for when a = 1, b = 1
```

For our harmonization task we are going to create a list with the chords that we are going to be considering.

We are also going to assign values to all the combinations of two chords that we can have in our sequence. These values will be the values for the potentials.

Let's start by defiining the `chord_list`.

In [105]:
chords_list= ['I','ii','iii','IV','V','vi','viid']

We'll create a dictionary named potentials, whose keys will pairs of variables representing the chord at each time point, i.e. $\mathtt{I}_i, \mathtt{ii}_i, \mathtt{iii}_i, \mathtt{IV}_i, \mathtt{V}_i, \mathtt{vi}_i, \mathtt{VIIdim}_i$ where $i\in [4]$

In [106]:
potentials = {} 

We will use the following values to avoid having multiple chords at a time point. (Table 7)

In [107]:
not_simultaneous = {(0, 0): 50,(0, 1): 50,(1, 0): 50,(1, 1): 100} # to avoid having two simultaneous chords

We define the potentials to be used when both variables are `(1,1)`. (Table 8 and Table 9)

In [108]:
values={('I','I'):100, ('I','ii'):50,('I','iii'):50,('I','IV'):50, ('I','V'):50, ('I','vi'):50, ('I','viid'):50,
             ('ii','I'):50, ('ii','ii'):100,('ii','iii'):50,('ii','IV'): 50, ('ii','V'): 50, ('ii','vi'):50, ('ii','viid'):50,
             ('iii','I'):50, ('iii','ii'):50,('iii','iii'):100,('iii','IV'):50, ('iii','V'):50, ('iii','vi'):50, ('iii','viid'):50,
             ('IV','I'):50, ('IV','ii'):50,('IV','iii'):50,('IV','IV'):100, ('IV','V'):50, ('IV','vi'):50, ('IV','viid'):50,
             ('V','I'):0, ('V','ii'):50,('V','iii'):50,('V','IV'):50, ('V','V'):50, ('V','vi'):50, ('V','viid'):50,
             ('vi','I'):50, ('vi','ii'):50,('vi','iii'):50,('vi','IV'):50, ('vi','V'):50, ('vi','vi'):100, ('vi','viid'):50,
             ('viid','I'):50, ('viid','ii'):50,('viid','iii'):50,('viid','IV'):50, ('viid','V'):50, ('viid','vi'):50, ('viid','viid'):100
            }

Finally we fill the potentials dictionary. Note that we need to consider combinations of all time points and chords.

In [109]:
for i, j in product(range(1,4),chords_list): # in this example we consider a sequence of four chords only
    for m in chords_list:
        index = chords_list.index(m)
        inner_list = chords_list[index+1:(len(chords_list))]
        for n in inner_list:
            potentials[(f"{i}_{m}",f"{i}_{n}")]= not_simultaneous # to avoid simultaneous chords
    for k in chords_list:
            potentials[(f"{i}_{j}",f"{i+1}_{k}")]={(0, 0): 50,(0, 1): 50,(1, 0): 50,(1, 1): values[(j,k)]}

In the previous cell we have considered the last chord as i was restricted to 1,2,3. We avoid having muliple chords as the last chord using the following cell. 

In [110]:
last_chord = 4
for j in chords_list:
    index = chords_list.index(j)
    inner_list = chords_list[index+1:(len(chords_list))]
    for k in inner_list:
        potentials[(f"{last_chord}_{j}",f"{last_chord}_{k}")] = not_simultaneous # to avoid simultaneous chords

<b>Running the experiment

We first create a markov network using the networx. 

In [119]:
MN = dnx.markov_network(potentials)  # create a Markov Network from the potentials
bqm = dnx.markov_network_bqm(MN)  # convert the Markov Network to a BQM

Next, we create the sample and use `sample_markov_network` function. We can use the `fixed_variables` parameter to set certain variables. In this example, we fix the first and the last chords. 

In [120]:
# let's consider a 4 chords progression starting in I and ending in I
bqm.fix_variable('1_I', 1)  # fix the first chord to 'I'
bqm.fix_variable('4_I', 1)  # fix the last chord to 'I' 
sampleset, energy = DYNEX(bqm, num_reads=100000, annealing_time=1000)

[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬───────────────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CIRCUITS   │ STEPS   │ GROUND STATE   │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼───────────────────────────┼────────────┼─────────┼────────────────┤
│         -1 │       34 │             117 │           0 │           │ *** WAITING FOR READS *** │            │         │                │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴───────────────────────────┴────────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.29 SECONDS
[DYNEX] SAMPLESET READY


<b> Interpreting the results

Let's check how the sample looks like. At a certain time point, there should be exactly one variable which is set to 1.

In [124]:
print(sampleset)

dict_items([('1_IV', 0), ('1_V', 0), ('1_ii', 0), ('1_iii', 0), ('1_vi', 0), ('1_viid', 0), ('2_I', 0), ('2_IV', 0), ('2_V', 0), ('2_ii', 0), ('2_iii', 0), ('2_vi', 0), ('2_viid', 0), ('3_I', 0), ('3_IV', 0), ('3_V', 1), ('3_ii', 0), ('3_iii', 0), ('3_vi', 0), ('3_viid', 0), ('4_IV', 0), ('4_V', 0), ('4_ii', 0), ('4_iii', 0), ('4_vi', 0), ('4_viid', 0)])


The variables that are set to 1 will give us the chord sequence we are looking for. 

In [126]:
chord_sequence = [k for k,v in sampleset.items() if v==1]
print(chord_sequence)

['3_V']


`sample` is a list containing only a single sample and we can directly store it using pickle.

In [128]:
filename = f"experiment7"
with open(filename, 'wb') as handle:
    pickle.dump(sampleset, handle)

---

### Example 8 (Page 22) - Harmonizing melody

<b> Creating the model

The list `P` represents the degrees of the scale

In [129]:
P = [1,2,3,4,5,6,7,8]
n = 20

The `note_list` represents the music pieace we would like to harmonize.

In [130]:
note_list = [1, 4, 3, 5, 1, 7, 8, 3, 5, 4, 3, 5, 4, 3, 6, 5, 4, 3, 5, 1]

We would like to generate a triad. Hence, as opposed to single note constraint, we have the constraint forcing that there are three notes at each time point $
\sum_{j \in P} x_{i,j} = 3
$. (Eq.41)



In [131]:
H = 0
for i in range(1,n+1):
    H += 2*Constraint((3-sum(Binary(f"x_{i}_{p}") for p in P))**2, f"Triad_{i}")

We would like to have the first and the last triad to be built on the first degree of the scale, hence we use the following constraints. (Eq. 42)

In [132]:
H += Constraint((1-Binary(f"x_{1}_{3}"))**2, f"note_{1}_3")
H += Constraint((1-Binary(f"x_{1}_{5}"))**2, f"note_{1}_5")
H += Constraint((1-Binary(f"x_{20}_{3}"))**2, f"note_{20}_3")
H += Constraint((1-Binary(f"x_{20}_{5}"))**2, f"note_{20}_5")
H += Constraint((1-Binary(f"x_{20}_{1}"))**2, f"note_{20}_1")

Each triad should contain the note given in the `note_list`.

In [133]:
for i in range(1,n+1):
    H += Constraint((1-Binary(f"x_{i}_{note_list[(i-1)]}"))**2, f"note_{i}")

We would like to avoid consecutive degrees to appear in the triad, as well as degrees 1-7, 2-6, 1-8. Hence we add the following penalty terms to the objective. (Eq. 44)

In [134]:
A = [1,6,7]
for i in range(1,n+1):
    for j, k in product(P,P):
        if abs(j-k) in A:
            H += Binary(f"x_{i}_{j}")*(Binary(f"x_{i}_{k}"))

<b> Running the experiment

In [135]:
model = H.compile()
qubo, of = model.to_qubo()
bqm = dimod.BinaryQuadraticModel.from_qubo(qubo,of)

In [136]:
sampleset, energy = DYNEX(bqm, num_reads=100000, annealing_time=1000)

[DYNEX] SAMPLER INITIALISED
[DYNEX|TESTNET] *** WAITING FOR READS ***
╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬───────────────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │ ELAPSED   │ WORKERS READ              │ CIRCUITS   │ STEPS   │ GROUND STATE   │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼───────────────────────────┼────────────┼─────────┼────────────────┤
│         -1 │      168 │             728 │           0 │           │ *** WAITING FOR READS *** │            │         │                │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴───────────────────────────┴────────────┴─────────┴────────────────╯

[DYNEX] FINISHED READ AFTER 0.30 SECONDS
[DYNEX] SAMPLESET READY


<b> Interpreting the results

We use the method `translate_chord` to convert our sample into a list of chords.

In [138]:
seq = translate_chord(sampleset,P,n)
print(seq)

{1: [1, 3, 5], 2: [4, 6, 8], 3: [3, 5, 7], 4: [1, 3, 5], 5: [1, 3, 6], 6: [3, 5, 7], 7: [3, 6, 8], 8: [1, 3, 5], 9: [3, 5, 7], 10: [1, 4, 6], 11: [3, 5, 8], 12: [3, 5, 7], 13: [4, 6, 8], 14: [1, 3, 6], 15: [1, 3, 6], 16: [1, 3, 6], 17: [1, 4, 6], 18: [3, 6, 8], 19: [3, 5, 8], 20: [1, 3, 5]}


You can check if any of the constraints are broken or not.

In [139]:
dec = model.decode_sample(sampleset, vartype='BINARY')
print(dec.constraints(only_broken=True))

{'note_16': (False, 1.0)}


In [140]:
filename = f"experiment8"
solution = dimod.SampleSet.from_samples(sampleset, energy=energy, vartype=dimod.BINARY)
store_results(filename, solution)